In [8]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns

# 1. GIỚI THIỆU & CÂU HỎI

YouTube (trang web chia sẻ video nổi tiếng thế giới) duy trì danh sách các video thịnh hành nhất trên nền tảng này. Theo tạp chí Variety, “Để xác định video thịnh hành nhất của năm, YouTube sử dụng kết hợp nhiều yếu tố bao gồm đo lường tương tác của người dùng (số lượt xem, lượt chia sẻ, nhận xét và lượt thích). Những nghệ sĩ biểu diễn hàng đầu trong danh sách thịnh hành của YouTube là các video ca nhạc (chẳng hạn như "Gangam Style" nổi tiếng), người nổi tiếng và / hoặc các buổi biểu diễn truyền hình thực tế và các video lan truyền ngẫu nhiên của anh chàng có máy ảnh mà YouTube nổi tiếng.

## Câu hỏi:


# 2. ĐỌC DỮ LIỆU

In [9]:
data_df = pd.read_csv("USvideos.csv")

##  Thông tin về dữ liệu

Tập dữ liệu này là bản ghi hàng ngày về các video thịnh hành nhất trên YouTube mà sử dụng cho lab này là thống kê của US. Được tác giả thu thấp thông qua Youtube API.

Link sưu tầm: https://www.kaggle.com/datasnaek/youtube-new
+ Usage Information
    - License: CC0 Public Domain
    - Visibility:       Public
+ Maintainers
    - Dataset owner: Mitchell J
+ Updates
    - Expected update frequency: Not specified
    - Last updated: 2019-06-03
    - Date created: 2017-11-14
    - Current version: Version 115

# 3. KHÁM PHÁ DỮ LIỆU

In [10]:
data_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [11]:
data_df.shape

(40949, 16)

+ Số dòng: 40949
+ Số cột: 16

## Ý nghĩa của mỗi thuộc tính:
+ video_id: biến định danh video
+ trending_date: ngày video lên top trending
+ title: tựa đề của video
+ channel_title: kênh đăng tải video lên Youtube
+ category_id
+ publish_time: ngày video được push lên Youtube
+ tags: 
+ views: lượng xem tính đến ngày lên top
+ likes: lượng thích tính đến ngày lên top
+ dislikes: lượng không tính đến ngày lên top
+ comment_count: số bình luận
+ thumbnail_link: 
+ comments_disabled: 
+ ratings_disabled: 
+ video_error_or_removed: video có lỗi hay bị xóa đi(True nếu có lỗi hoặc bị xóa và ngược lại)
+ description: phần mô tả cho bài đăng

## Mỗi cột có dtype là gì:

In [12]:
col_dtypes=pd.Series(data_df.dtypes,index=data_df.columns)
col_dtypes

video_id                  object
trending_date             object
title                     object
channel_title             object
category_id                int64
publish_time              object
tags                      object
views                      int64
likes                      int64
dislikes                   int64
comment_count              int64
thumbnail_link            object
comments_disabled           bool
ratings_disabled            bool
video_error_or_removed      bool
description               object
dtype: object

## Cột có dtype là object nghĩa là sao?

# 4. TIỀN XỬ LÍ